# 心理咨询论坛问答语料 - 制作迁徙学习数据集

为 NVIDIA/Megatron-LM 准备该领域的 finetune 数据集

## 要点

- 使用教少量的心理咨询论坛问答数据，在之前的泛情绪类预训练模型基础上进行 finetune
- 使用特殊标记(Token) 分隔问题标题、文本、答案文本

## Importings

In [1]:
import os
import json
import random
from copy import copy
from contextlib import closing, ExitStack
from functools import partial
from glob import glob, iglob
from fileinput import FileInput
from itertools import chain, repeat
from multiprocessing import Pool

import pandas as pd
import sentencepiece as spm
from tqdm.auto import tqdm

from IPython.display import display

## Tokenizer

使用与 Pretrained-Model 同样的 tokenizer

In [2]:
spm_model_path = '../data/spm/gpt2_huamei_corpus_bpe_32k_v2.model'

tokenizer = spm.SentencePieceProcessor()
tokenizer.load(spm_model_path)

True

### 特殊标记

该 Tokenizer 既有的特殊标记有：

In [3]:
for i in range(8):
  print(i, tokenizer.id_to_piece(i), tokenizer.is_control(i))

0 <pad> True
1 <unk> False
2 <bos> True
3 <eos> True
4 <sep> False
5 <cls> False
6 <|endoftext|> False
7 一个 False


我们选用：

- 连续两个`<sep>` 表示问题与回答的间隔
- 一个 `<sep>` 表示问题中，标题、正文、类型的分隔
- 每个类型标签文本的开头，加上`<cls>`

格式是：

```
<title-of-question><sep><text-of-question><seq>[[<cls>category1,][[<cls>category2,] ... ]<sep><sep><text-of-answer>
```

例如：

```
脸上长痘痘怎么办？<sep><cls>健康<cls>面部护理<sep>我18岁，脸上有很多豆豆，流脓，老不好，吃了很多药，怎么办？<sep><sep>看了楼主的问题，我感动深受，建立你来苗医生！
```

In [4]:
SEP = tokenizer.id_to_piece(4)
CLS = tokenizer.id_to_piece(5)

print(
    SEP,
    CLS
)

<sep> <cls>


## Corpus

In [5]:
corpus_files = glob('/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/*.json')

In [6]:
print('语料文件:')
display(corpus_files)
print()

it = FileInput(corpus_files)
total = sum(1 for _ in tqdm(it, unit='line'))

print(f'总行数: {total:,d}')

语料文件:


['/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/valid_xinli_qax.json',
 '/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/train_xinli_qax.json',
 '/home/Public/data/transfer-learning/output/output-qa/xinli001_jiandanxinli-qa.topics/test_xinli_qax.json']


总行数: 187,156


现在预览语料数据格式，随机选择一个：

In [29]:
n = random.randint(0, total)

it = FileInput(corpus_files)
for i, line in tqdm(zip(range(n), it), total=n):
    if i+1 < n:
        continue
    data = json.loads(line)
    display(i)
    display(data)


108936

{'topics': ['人格特质', '情绪', '情绪调节'],
 'title': ['因为减肥而带来心理疾病，该怎么办？'],
 'text': ['可能从16岁开始吧，我就对减肥很执着，一直都在努力减肥，现在163/52kg想着继续减肥，继母家里的人从小都说我很胖，我心里很受打击，就一直在减肥，最近一年吧就开始发现自己心理好像越来越不正常了，吃饭的时候不敢吃，吃完了就很难过觉得自己会长胖，然后就进入了节食的状态，节食了两三天后又暴食，吃到胃很撑很难受的那种，然后心情就会很难过，很讨厌自己为什么吃这么多，不吃饭的时候自己也在反思说为什么要这样，为什么要节食，为什么要减肥',
  '现在很害怕吃东西，也很害怕看到食物，怕自己忍不住往嘴里塞东西，以前吃东西感觉到开心，现在不知道什么是开心，对什么事情都提不起兴趣，很不喜欢这样的自己，不知道该怎么办'],
 'answers': [['您好，',
   '看您的描述，',
   '已经不单只是减肥的问题了，',
   '应该属于临床上比较常见的体象障碍和神经性贪食、厌食症，',
   '建议到医院确诊一下，',
   '同时配合心理治疗，',
   '才有较好的效果。',
   '另外，',
   '神经症厌食在精神分析的理解框架下，',
   '并不是什么都不想吃、不能吃，',
   '而是TA吃了一个“空”。',
   '这个“空”是由无意识、幻想所构建的，',
   '是来源于重要养育人的话语下所构建的症状，',
   '背后是一个幻想的支撑。',
   '而心理咨询的工作，',
   '就是陪伴您去穿越这个症状式的幻想，',
   '达到主体生命的解放。',
   '当然，',
   '上面所提到的只是一点点理论，',
   '同时标签也没有任何意义，',
   '重要的是接受一段时间的心理咨询，',
   '您会慢慢看到自己症状背后的幻想逻辑，',
   '正是这些幻想导致减肥执念在不断重复。'],
  ['千万不要节食，', '不要让自己处于饥饿的感觉中，', '平常感觉不饿就可以'],
  ['听起来，',
   '你在进食方面出现了困扰，',
   '我想除了可能的心理原因之外，',
   '但是进食问题会影响身体健康，',
   '会影响你对食物的态度，',
   '如果你有条件的话，',
   

## 按主题搜索

试试看，指定一个标签，看看有多少个样本

In [18]:
topic = '婚姻'

def filter_line_by_topic(line, topic):
    line = line.strip()
    if line:
        sample = json.loads(line)
        topics = sample.get('topics', [])
        if topic in topics:
            return True
    return False

it_lines = FileInput(corpus_files)
with Pool() as pool:
    it_map = pool.imap_unordered(
        partial(filter_line_by_topic, topic=topic),
        tqdm(it_lines, total=total),
        chunksize=min(1024, total//os.cpu_count()+1)
    )
    it_map = tqdm(it_map, total=total)
    c = sum(1 for x in it_map if x)

print(f'主题 {topic!r} 数量: {c:,d}, 占比: {c/total}')



主题 '婚姻' 数量: 31,760, 占比: 0.16969800594156748


## 样本选用和格式转换

我们应：

- 选取问题+回答构成输入样本
- 一个问题可以对应多个回答
- 不要太短的
- 不要太长的

在此基础上，选用 `10,000` 个样本用于今次的开发

In [51]:
### 长短限制定义：

MIN_QUESTION_LENGTH = 16
MAX_QUESTION_LENGTH = 1024
MIN_ANSWER_LENGTH = 32
MAX_LENGTH = 2048


def extract_samples(line):
    line = line.strip()
    if not line:
        return []
    data = json.loads(line)
    result = []
#     # 主题不对的不要！
    topics = data.get('topics', []) 
#     if topic not in topics:
#         return []
#     # 没有主题的不要
#     if not topics:
#         return []
    # 没有回答的不要
    if not data.get('answers'):
        return []
    # 问题:
    title = ''.join(data['title']).strip()
#     # 没有标题的不要
#     if not title:
#         return []
    text = ''.join(data['text']).strip()
#     # 没有问题内容的不要
#     if not text:
#         return []
    n_title = len(tokenizer.encode_as_ids(title))
    n_text = len(tokenizer.encode_as_ids(text))
    # 太短不要
    if n_title + n_text < MIN_QUESTION_LENGTH:
        return []
    # 太长不要
    if n_title + n_text > MAX_QUESTION_LENGTH:
        return []
    #
    question_string = title + SEP + text
    if topics:
        question_string += SEP
        for topic in topics:
            question_string += CLS + topic.strip()
    # 答案
    for answer in data.get('answers', []):
        text = ''.join(answer).strip()
        length = len(tokenizer.encode_as_ids(text))
        # 太短不要
        if length < MIN_ANSWER_LENGTH:
            continue
        # 太长不要
        if n_title + n_text + length > MAX_LENGTH:
            continue
        result.append(question_string + SEP*2 + text)
    #
    return result

用上一次随机取得的行试试看：

In [52]:
extract_samples(line)

['因为减肥而带来心理疾病，该怎么办？<sep>可能从16岁开始吧，我就对减肥很执着，一直都在努力减肥，现在163/52kg想着继续减肥，继母家里的人从小都说我很胖，我心里很受打击，就一直在减肥，最近一年吧就开始发现自己心理好像越来越不正常了，吃饭的时候不敢吃，吃完了就很难过觉得自己会长胖，然后就进入了节食的状态，节食了两三天后又暴食，吃到胃很撑很难受的那种，然后心情就会很难过，很讨厌自己为什么吃这么多，不吃饭的时候自己也在反思说为什么要这样，为什么要节食，为什么要减肥现在很害怕吃东西，也很害怕看到食物，怕自己忍不住往嘴里塞东西，以前吃东西感觉到开心，现在不知道什么是开心，对什么事情都提不起兴趣，很不喜欢这样的自己，不知道该怎么办<sep><cls>人格特质<cls>情绪<cls>情绪调节<sep><sep>您好，看您的描述，已经不单只是减肥的问题了，应该属于临床上比较常见的体象障碍和神经性贪食、厌食症，建议到医院确诊一下，同时配合心理治疗，才有较好的效果。另外，神经症厌食在精神分析的理解框架下，并不是什么都不想吃、不能吃，而是TA吃了一个“空”。这个“空”是由无意识、幻想所构建的，是来源于重要养育人的话语下所构建的症状，背后是一个幻想的支撑。而心理咨询的工作，就是陪伴您去穿越这个症状式的幻想，达到主体生命的解放。当然，上面所提到的只是一点点理论，同时标签也没有任何意义，重要的是接受一段时间的心理咨询，您会慢慢看到自己症状背后的幻想逻辑，正是这些幻想导致减肥执念在不断重复。',
 '因为减肥而带来心理疾病，该怎么办？<sep>可能从16岁开始吧，我就对减肥很执着，一直都在努力减肥，现在163/52kg想着继续减肥，继母家里的人从小都说我很胖，我心里很受打击，就一直在减肥，最近一年吧就开始发现自己心理好像越来越不正常了，吃饭的时候不敢吃，吃完了就很难过觉得自己会长胖，然后就进入了节食的状态，节食了两三天后又暴食，吃到胃很撑很难受的那种，然后心情就会很难过，很讨厌自己为什么吃这么多，不吃饭的时候自己也在反思说为什么要这样，为什么要节食，为什么要减肥现在很害怕吃东西，也很害怕看到食物，怕自己忍不住往嘴里塞东西，以前吃东西感觉到开心，现在不知道什么是开心，对什么事情都提不起兴趣，很不喜欢这样的自己，不知道该怎么办<sep><cls>人格特质<cls>情绪<cls>情绪调节<

## 计算全部样本

In [53]:
def generate_samples_from_corpus():
    lines_iter = FileInput(corpus_files)
    lines_iter = tqdm(lines_iter, desc='MAP', total=total)
    with Pool() as pool:
        mapping_iter = pool.imap_unordered(extract_samples, lines_iter, chunksize=min(1024, total//os.cpu_count())+1)
        mapping_iter = tqdm(mapping_iter, desc='RDC', total=total)
        yield from chain.from_iterable(mapping_iter)

print('sample ...')        
%time samples = list(generate_samples_from_corpus())

print('shuffle ...')
%time random.shuffle(samples)

print(f'满足要求的样本数: {len(samples):,d}')

sample ...




CPU times: user 5.48 s, sys: 2.69 s, total: 8.17 s
Wall time: 8.8 s
shuffle ...
CPU times: user 280 ms, sys: 11 ms, total: 291 ms
Wall time: 290 ms
满足要求的样本数: 304,218


随机显示一个:

In [44]:
s = random.choice(samples)
print(s)

总是多想<sep>我是一个很敏感的人，别人和我之间相处时发生的事，我却一直放不下，总是在多想，结果总是越想越焦躁。有时候明明只是一件小事，但我总是爱多想很久，很影响生活。心里烦闷的很，不知道怎么样才能让自己不多心多想放下这些事。<sep><cls>人格特质<cls>情绪<cls>焦虑<sep><sep>你好，不知道你多大了？看起来你对自己是有觉察的，敏感让你焦虑，并影响到了自己的生活和心情。你的敏感源于你内心的自卑，你能选择来这里求助，就非常的不简单。咨询是一个过程，我需要了解更多的情况，才能帮你分析解决问题。我帮助过很多和你一样困扰的人，相信也可以帮到你。可以多说一些你的情况吗？


## 采样并存储

### 划分数据集

划分为 train/val/test

In [57]:
parts = {
    'train': 300_000,
    'val': 2_000,
    'test': 2_000,
}

print('数据集划分：')
display(parts)

assert sum(parts.values()) <= len(samples)

数据集划分：


{'train': 300000, 'val': 2000, 'test': 2000}

### 随机采样

随机种子:

In [58]:
%%time

seeds = []
seeds.extend(chain.from_iterable(
    repeat(k, v) for k, v in parts.items()
))
seeds.extend(repeat(None, len(samples)-sum(parts.values())))
random.shuffle(seeds)

assert len(seeds) == len(samples)

CPU times: user 270 ms, sys: 0 ns, total: 270 ms
Wall time: 269 ms


### 保存结果

#### 输出文件路径

保存为 NVIDIA/Megatron-LM 的 Loose Json 格式，使用 `"text"` 作为 Key。


In [59]:
%%time

# from slugify import slugify

# topic_slug = slugify(topic, separator='')

# output_files = {
#     k: os.path.join('..', 'data', f'xinliqa_{topic_slug}.{k}.json')
#     for k in part_num.keys()
# }

# print('输出文件:')
# display(output_files)

output_dir = '../data/xinliqa'

os.makedirs(output_dir, exist_ok=True)

output_files = {
    k: os.path.join(output_dir, f'xinliqa.{k}.json')
    for k in parts.keys()
}

print('输出文件:')
display(output_files)

输出文件:


{'train': '../data/xinliqa/xinliqa.train.json',
 'val': '../data/xinliqa/xinliqa.val.json',
 'test': '../data/xinliqa/xinliqa.test.json'}

CPU times: user 7.08 ms, sys: 0 ns, total: 7.08 ms
Wall time: 4.67 ms


#### 写目标文件

In [60]:
lines_iter = FileInput(corpus_files)

with ExitStack() as stack:
    files = {
        k: stack.enter_context(open(v, 'w'))
        for k, v in output_files.items()
    }
    for seed, string in tqdm(zip(seeds, samples), total=len(samples)):
        fp = files.get(seed)
        if not fp:
            continue
        print(
            json.dumps({'text': string}, ensure_ascii=False),
            file=fp
        )

检查输出文件

In [61]:
s = ' '.join(output_files.values())

!wc -l {s}
print()

!du -h {s}
print()

   300000 ../data/xinliqa/xinliqa.train.json
     2000 ../data/xinliqa/xinliqa.val.json
     2000 ../data/xinliqa/xinliqa.test.json
   304000 总用量

352M	../data/xinliqa/xinliqa.train.json
2.4M	../data/xinliqa/xinliqa.val.json
2.4M	../data/xinliqa/xinliqa.test.json

